In [2]:
import duckdb
from minio import Minio

# Connect to MinIO
minio_client = Minio(
    "localhost:9000",
    access_key="minioadmin",
    secret_key="minioadmin",
    secure=False
)

# Configure DuckDB for S3 access
conn = duckdb.connect()
conn.execute("""
        INSTALL httpfs;
        LOAD httpfs;
        SET s3_endpoint='minio:9000';
        SET s3_access_key_id='minioadmin';
        SET s3_secret_access_key='minioadmin';
        SET s3_use_ssl=false;
        SET s3_url_style='path';
""")

# Query data from MinIO
result = conn.execute("""
    SELECT * 
    from read_parquet(
            's3a://blake/silver/breweries/**/*.parquet',
            hive_partitioning=1
        )
    LIMIT 10
""").fetchdf()

print(result)

                   address_1 address_2 address_3 brewery_type  \
0           Schleppe-Platz 1      None      None        large   
1            Brauhausgasse 6      None      None        large   
2                      L67 7      None      None         nano   
3  Mautner-Markhof-Stra�e 11      None      None        large   
4        Dr.-Beurle-Stra�e 1      None      None        large   
5              Stiftstra�e 6      None      None         nano   
6                    Zipf 22      None      None        large   
7          Triester Str. 361      None      None        large   
8       Feldkirchenstra�e 40      None      None          bar   
9            Brauhausgasse 1      None      None        large   

                                     id            latitude  \
0  c88759b6-88d6-4172-bed7-61dec61e50aa    46.6416966591789   
1  f51b3f3d-be14-40da-914a-3b5c83f75bbf   46.61662463422168   
2  3fe1de22-c30c-446d-b2f2-913ca9f6378e   46.83992642711985   
3  4c8dff75-fa8c-41ce-9999-1f063